# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


Ambiente locale rilevato.


# Dataset Setup

In [2]:
from utils import setup_project
# Ora puoi passare agli import del modello
from dataset.capitain_cook_4d_mlp_dataset import CaptainCook4DMLP_Dataset, DatasetSource
from models.BaselineV1_MLP import BaselineV1_MLP

# Esegue: Setup Dati (unzip/copy), Login WandB, Setup Device
device = setup_project.initialize(ROOT_DIR)
DATASET_SOURCE = DatasetSource.OMNIVORE

# Import wandb
import wandb

Setup Progetto in: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection
source_path: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
Setup Dati da: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
Inizio setup dati...
   Sorgente: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\_file
   Destinazione: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data
Copia cartella: annotation_json...
Copia cartella: omnivore...
✅ Setup completato! Dati pronti in: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\marco\_netrc
wandb: Currently logged in as: s339450 (s339450-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB Logged in.
Device: cuda


# Configuration

In [3]:
# Configurazione esperimento
config = {
    "architecture": "BaselineV1_Transformer_" + DATASET_SOURCE.value,
    "dataset": "CaptainCook4D",
    "feature_extractor": DATASET_SOURCE.value,
    "input_dim": DATASET_SOURCE.input_dims(),
    "batch_size": 512,
    "learning_rate": 0.0001,
    "epochs": 100,
    "pos_weight": 1.5,
    "optimizer": "Adam",
    "loss_function": "BCEWithLogitsLoss",
    "seed": 42
}

# Dataset Split

In [4]:
import os
from dataset.capitain_cook_4d_mlp_dataset import DatasetSource
from dataset.capitain_cook_4d_transformer_dataset import CaptainCook4DTransformer_Dataset
from dataset.utils import get_tranformer_loaders

try:
    full_dataset = CaptainCook4DTransformer_Dataset(
        dataset_source=DATASET_SOURCE, 
        root_dir=ROOT_DIR
    )
    train_loader, val_loader, test_loader = get_tranformer_loaders(
        full_dataset,
        batch_size=config["batch_size"],
        seed=config["seed"]
    )

except Exception as e:
    print(f"❌ Errore: {e}")

Loading from: c:\Users\marco\Desktop\Marco\Programmazione\C\EsPoli\Advanced Machine Learning\MistakeDetection\data\omnivore...
[V2] Numero di step univoci: 200

DATASET INFO [V2 - STEP-BASED]
   Total Steps: 200
   Total Sub-seconds: 8828
   Avg seconds per step: 44.14
FULL DATASET       | Steps: 200  | Subsecs: 8828  | OK: 4917  (55.7%) | ERR: 3911  (44.3%) | Ratio: 1:1.3
-------------------------------------------------------------------------------------
TRAIN SET          | Steps: 140  | Subsecs: 6252  | OK: 3281  (52.5%) | ERR: 2971  (47.5%) | Ratio: 1:1.1
VALIDATION SET     | Steps: 20   | Subsecs: 799   | OK: 491   (61.5%) | ERR: 308   (38.5%) | Ratio: 1:1.6
TEST SET           | Steps: 40   | Subsecs: 1777  | OK: 1145  (64.4%) | ERR: 632   (35.6%) | Ratio: 1:1.8

[V2] Batch size forzato a 1 (uno step per batch)
[V2] Training loop itererà su 140 step



In [5]:
# V2: quando accedi a dataset[idx], dove idx è l'indice dello STEP

full_dataset.print_item(0)


V2 DATASET ITEM [0]
Step ID:              1_10_0_11.749_46.437
Sequence length:      36 seconds
Features shape:       torch.Size([36, 1024]) (seconds x features)
Labels shape:         torch.Size([36]) (seconds)

Step details:
  Video ID:             1_10
  Step index:           0
  Timing:               11.749s - 46.437s (34.688s)
  Label sequence:       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
